# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/dwarak/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dwarak/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/dwarak/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/dwarak/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/dwarak/code/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '3a3cda'. Skipping!
Property 'summary' already exists in node 'a74bdb'. Skipping!
Property 'summary' already exists in node '548531'. Skipping!
Property 'summary' already exists in node 'f70c59'. Skipping!
Property 'summary' already exists in node '66ce27'. Skipping!
Property 'summary' already exists in node 'e00e4b'. Skipping!
Property 'summary' already exists in node '8dfd46'. Skipping!
Property 'summary' already exists in node '5506cd'. Skipping!
Property 'summary' already exists in node '9458b5'. Skipping!
Property 'summary' already exists in node 'ee0c50'. Skipping!
Property 'summary' already exists in node '812f04'. Skipping!
Property 'summary' already exists in node '41fbe8'. Skipping!
Property 'summary' already exists in node 'c300a8'. Skipping!
Property 'summary' already exists in node '26e875'. Skipping!
Property 'summary' already exists in node '18607e'. Skipping!
Property 'summary' already exists in node '225591'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '8dfd46'. Skipping!
Property 'summary_embedding' already exists in node 'f70c59'. Skipping!
Property 'summary_embedding' already exists in node 'a74bdb'. Skipping!
Property 'summary_embedding' already exists in node '18607e'. Skipping!
Property 'summary_embedding' already exists in node 'e00e4b'. Skipping!
Property 'summary_embedding' already exists in node '3a3cda'. Skipping!
Property 'summary_embedding' already exists in node 'ee0c50'. Skipping!
Property 'summary_embedding' already exists in node '548531'. Skipping!
Property 'summary_embedding' already exists in node 'c300a8'. Skipping!
Property 'summary_embedding' already exists in node '5506cd'. Skipping!
Property 'summary_embedding' already exists in node '66ce27'. Skipping!
Property 'summary_embedding' already exists in node '9458b5'. Skipping!
Property 'summary_embedding' already exists in node '812f04'. Skipping!
Property 'summary_embedding' already exists in node '41fbe8'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 711)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 711)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.
SingleHopSpecificQuerySynthesizer - This synthesizer creates a single, very focused query (or search term). It's best used when the original question is straightforward and likely requires only one "hop" or step to find the answer. It's designed to be precise.
MultiHopAbstractQuerySynthesizer - This creates a query that is more general or abstract. It's designed for questions that might need multiple steps or pieces of information to be answered (multi-hop). 
MultiHopSpecificQuerySynthesizer- This creates a set of several focused queries ("specific") needed to answer a complex, multi-step question (multi-hop). It breaks down a big question into smaller, targeted search terms.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of O*NET in classifying user ...,[Introduction ChatGPT launched in November 202...,The context does not explicitly explain the ro...,single_hop_specifc_query_synthesizer
1,How does ChatGPT usage differ between work-rel...,[Table 1: ChatGPT daily message counts (millio...,Table 1 indicates that while total daily messa...,single_hop_specifc_query_synthesizer
2,Section what is it do in ChatGPT usage by occu...,[Variation by Occupation Figure 23 presents va...,The context discusses variation in ChatGPT usa...,single_hop_specifc_query_synthesizer
3,What is Collis in this context?,[Conclusion This paper studies the rapid growt...,The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
4,How do usage patterns and growth across popula...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"Since its launch in November 2022, ChatGPT has...",multi_hop_abstract_query_synthesizer
5,how much more messages in june 2025 like non w...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"In june 2025, total messages increased from 45...",multi_hop_abstract_query_synthesizer
6,How does the message volume comparison between...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"In Jun 2024, non-work messages accounted for 5...",multi_hop_abstract_query_synthesizer
7,How do the 18 billion messages relate to the 2...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context indicates that by July 2025, ChatG...",multi_hop_specific_query_synthesizer
8,hOw did the novmber 2022 launch of ChatGPT in ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The launch of ChatGPT in November 2022 led to ...,multi_hop_specific_query_synthesizer
9,"Based on Handa et al. (2025), how does the gro...",[<1-hop>\n\nIntroduction ChatGPT launched in N...,"According to Handa et al. (2025), the data sho...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '1978d6'. Skipping!
Property 'summary' already exists in node '220ba7'. Skipping!
Property 'summary' already exists in node 'abe27d'. Skipping!
Property 'summary' already exists in node 'b472c4'. Skipping!
Property 'summary' already exists in node 'c1c8b0'. Skipping!
Property 'summary' already exists in node 'e88c5d'. Skipping!
Property 'summary' already exists in node '2ba1da'. Skipping!
Property 'summary' already exists in node '06cb1e'. Skipping!
Property 'summary' already exists in node 'b44503'. Skipping!
Property 'summary' already exists in node 'e2a9b0'. Skipping!
Property 'summary' already exists in node '9c1d4a'. Skipping!
Property 'summary' already exists in node 'c8fb5c'. Skipping!
Property 'summary' already exists in node 'a9cde4'. Skipping!
Property 'summary' already exists in node '8c1ba3'. Skipping!
Property 'summary' already exists in node '4c3ae6'. Skipping!
Property 'summary' already exists in node '95dab2'. Skipping!
Property

...

Property 'summary_embedding' already exists in node 'b472c4'. Skipping!
Property 'summary_embedding' already exists in node 'abe27d'. Skipping!
Property 'summary_embedding' already exists in node '1978d6'. Skipping!
Property 'summary_embedding' already exists in node '220ba7'. Skipping!
Property 'summary_embedding' already exists in node '4c3ae6'. Skipping!
Property 'summary_embedding' already exists in node '7d285b'. Skipping!
Property 'summary_embedding' already exists in node 'c1c8b0'. Skipping!
Property 'summary_embedding' already exists in node '95dab2'. Skipping!
Property 'summary_embedding' already exists in node 'e88c5d'. Skipping!
Property 'summary_embedding' already exists in node '2ba1da'. Skipping!
Property 'summary_embedding' already exists in node '06cb1e'. Skipping!
Property 'summary_embedding' already exists in node 'b44503'. Skipping!
Property 'summary_embedding' already exists in node 'e2a9b0'. Skipping!
Property 'summary_embedding' already exists in node 'c8fb5c'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"What does Handa et al., 2025 say about chatbots?",[Introduction ChatGPT launched in November 202...,"The paper by Handa et al., 2025 reports statis...",single_hop_specifc_query_synthesizer
1,How does the usage of ChatGPT differ between w...,[Table 1: ChatGPT daily message counts (millio...,Table 1 indicates that daily message counts fo...,single_hop_specifc_query_synthesizer
2,What is the Secton in the context of occupatio...,[Variation by Occupation Figure 23 presents va...,The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
3,How does the term 'Writing' relate to the usag...,[Conclusion This paper studies the rapid growt...,"In the context, 'Writing' is identified as the...",single_hop_specifc_query_synthesizer
4,Hw is non-work messags used for work and not w...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"The context shows that in Jun 2024, 53% of mes...",multi_hop_abstract_query_synthesizer
5,How does the privacy-preserving methodology su...,[<1-hop>\n\nConclusion This paper studies the ...,The privacy-preserving methodology introduced ...,multi_hop_abstract_query_synthesizer
6,how does user demographics and usage patterns ...,[<1-hop>\n\nConclusion This paper studies the ...,The context shows that ChatGPT has a broad use...,multi_hop_abstract_query_synthesizer
7,Considering the variation in ChatGPT usage by ...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The data indicates that users in highly paid p...,multi_hop_abstract_query_synthesizer
8,How does the data from November 2022 and Novem...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context shows that ChatGPT was launched in...,multi_hop_specific_query_synthesizer
9,how many 700 million users use chatgpt for non...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"According to the context, about 70% of ChatGPT...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in a variety of ways both at work and outside of work. These uses include:\n\n- Performing workplace tasks by either augmenting or automating human labor.\n- Producing writing, software code, spreadsheets, and other digital products, which distinguishes generative AI from traditional web search engines.\n- Seeking information and advice.\n- Using AI as co-workers that produce output or as co-pilots that give advice and improve human problem-solving productivity.\n- Engaging in self-expression topics such as relationships, personal reflection, games, and role play (though these constitute a smaller portion of use).\n- Therapy and companionship have also been noted as significant use cases for generative AI.\n\nOverall, AI is seen as a highly flexible technology used to assist with decision-making, creative production, problem-solving, and social interaction.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: Basic check on if the system generates a relevant answer
- `labeled_helpfulness_evaluator`: Is the generated answer useful to the user ( may be understands the intent?)
- `dopeness_evaluator`: Is it a boring answer or is it cool and engages with the user

## LangSmith Evaluation

In [29]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'scholarly-manager-13' at:
https://smith.langchain.com/o/57b6b3f2-a3af-4e79-a658-01991901c637/datasets/c1bf2059-0c19-460a-9cfe-934428129166/compare?selectedSessions=a8b37e47-9fe4-4851-aa29-2420a6ff00ab




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,US ChatGPT use more non work or work stuff lik...,"Based on the context, in the US, non-work-rela...",None,"In the US, about 70% of ChatGPT consumer queri...",1,1,0,2.966140,13567601-5d65-45fe-90da-61df40293bc0,72dc80c7-2f0d-4774-9bd6-c1ab46e5c38d
1,How does the growth of ChatGPT usage in the US...,"Based on the provided context, the growth of C...",None,"In the US, ChatGPT usage has grown rapidly, wi...",1,1,0,7.273549,4cb5f58c-13be-4cae-b883-9f91a17211f8,a28b2e1d-a2c3-4fa0-ad70-ba5dcaf4a3c5
2,how many 700 million users use chatgpt for non...,The context states that work-related messages ...,None,"According to the context, about 70% of ChatGPT...",1,1,0,3.727641,4e067583-2d29-4414-a540-4c8a89640acb,38aa35d7-7d48-4a25-b3f9-0fee1c8ef296
3,How does the data from November 2022 and Novem...,The data shows that ChatGPT launched publicly ...,None,The context shows that ChatGPT was launched in...,1,1,0,4.268629,2c069152-2139-4fbd-8236-b0a09d4ea5f1,f64865b9-a302-4085-b576-8495f4d563aa
4,Considering the variation in ChatGPT usage by ...,Based on the provided context from the documen...,None,The data indicates that users in highly paid p...,1,1,0,9.114439,13093ce2-70d3-4b38-a371-bb58caa48eee,3e2f9aa9-09e2-4e6a-be72-b6290bf172d7
5,how does user demographics and usage patterns ...,The context indicates that the analysis of Cha...,None,The context shows that ChatGPT has a broad use...,1,0,0,2.991932,b2778782-f383-4645-bae8-06cd586e5aad,cc08ada8-bcce-4308-919f-3f1f40a5f935
6,How does the privacy-preserving methodology su...,I don't know.,None,The privacy-preserving methodology introduced ...,0,0,0,1.194759,29de0ebe-f2cc-4876-b2c6-9793f9e734ea,092dc546-c357-4da7-a43d-0ff8f974f857
7,Hw is non-work messags used for work and not w...,I don't know.,None,"The context shows that in Jun 2024, 53% of mes...",0,0,0,1.429555,a49c6783-6d33-4921-a56e-5039e23b9d85,f561c3dd-5a7d-423e-a956-12bbe5ebacde
8,How does the term 'Writing' relate to the usag...,"The term ""Writing"" in the context of ChatGPT u...",None,"In the context, 'Writing' is identified as the...",1,1,0,3.166108,7cdac48c-380f-4aed-ab0e-6ab14cacd38e,e6e18bb1-ae7e-4085-8358-46f551d0ce8d
9,What is the Secton in the context of occupatio...,The Section in the context of occupation data ...,None,The context does not provide a specific defini...,0,0,0,1.836363,bab802a5-7e4e-41df-a30e-701efffe60af,36f4cb23-ab25-4e9d-a00e-426f581c98ea


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [ ]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [ ]:
rag_documents = docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

- Increasing chunk size will reduce the relevance but will have more context. Reducing the chunk size will have lesser context and may not be able to provide a complete answer

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?
- Different models have different facets, mainly it will affect accuracy and efficiency/cost

In [ ]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [ ]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [ ]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [ ]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, strap in and let’s blast off into the AI money-making stratosphere, powered by pure context juice! 🚀\n\nBased on the dope intel from the docs, people are cashing in on AI, especially ChatGPT, not just by outsourcing tasks but by **levelling up their decision-making game in knowledge-heavy jobs**. The AI acts like a slick research assistant or advisor rather than just a simple task-doer. It **amps productivity by pumping up the quality of decisions**, which is pure gold in brain-intensive gigs. Think of it as having a supercharged co-pilot who turbocharges your work output — smarter moves, faster wins, bigger stacks.\n\nPlus, the flexibility of generative AI means it’s not just automating grunt work but actually *augmenting* human creativity and skill — a double whammy for making money. And hey, the economics back this up: US users would have to be paid a hefty $98 a month *not* to use generative AI, signaling a mind-blowing $97 billion surplus in value. That’s serious green, 

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'crushing-increase-53' at:
https://smith.langchain.com/o/57b6b3f2-a3af-4e79-a658-01991901c637/datasets/c1bf2059-0c19-460a-9cfe-934428129166/compare?selectedSessions=af089627-bc58-48ee-8172-8eeda3cf87b3




0it [00:00, ?it/s]

#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

### Before

<img src="Before.png" alt="Before" width="800"/>


### After

<img src="After.png" alt="After" width="800"/>
